In [1]:
# !pip3 install datacrunch

In [13]:
import os
from datacrunch import DataCrunchClient

# Get client secret from environment variable
CLIENT_SECRET = os.environ['DATACRUNCH_CLIENT_SECRET']
CLIENT_ID = os.environ['DATACRUNCH_CLIENT_ID']

# Create datcrunch client
datacrunch = DataCrunchClient(CLIENT_ID, CLIENT_SECRET)

In [14]:
import os
import subprocess
import shutil
email = 'poloniki@ya.ru'
key_filename = email.replace("@", "_").replace(".", "_")
def generate_ssh_key(email: str) -> str:
    try:
        # Create a filename based on the email
       
        
        # Define the command and its arguments as a list
        cmd = [
            "ssh-keygen",
            "-t", "ed25519",
            "-C", email,
            "-f", key_filename,  # Using the modified filename
            "-N", ""  # Empty passphrase
        ]
        
        # Run the command
        process = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        # Send an empty passphrase (newline) twice: once for the passphrase itself and once to confirm it
        process.communicate(input=b'\n\n')
        if process.returncode != 0:
            raise Exception("Error generating the key.")
        
        # Read the public key
        with open(f"{key_filename}.pub", "r") as f:
            pubkey = f.read()
        
        # Move generated keys to the ~/.ssh folder
        shutil.move(key_filename, os.path.expanduser(f"~/.ssh/{key_filename}"))
        shutil.move(f"{key_filename}.pub", os.path.expanduser(f"~/.ssh/{key_filename}.pub"))

        print(f"SSH key for {email} generated successfully!")
        return pubkey

    except Exception as e:
        print(f"Failed to generate SSH key for {email}. Reason: {e}")
        return None


In [15]:
# Get all SSH keys
your_email = "poloniki@ya.ru"
ssh_keys = datacrunch.ssh_keys.get()
ssh_keys = list(map(lambda key: key.id, ssh_keys))
if len(ssh_keys) == 0:
    public_key = generate_ssh_key(your_email)
    datacrunch.ssh_keys.create(your_email,public_key)
    ssh_keys = datacrunch.ssh_keys.get()
    ssh_keys = list(map(lambda key: key.id, ssh_keys))

In [16]:
# We select accessible GPU's that work really fast
instance_type=''
if datacrunch.instances.is_available(instance_type='1A6000.10V'):
    instance_type='1A6000.10V'
elif datacrunch.instances.is_available(instance_type='1A6000ADA.10V'):
    instance_type='1A6000ADA.10V'

In [11]:
# Read the Bash script's contents
with open('../scripts/clone_github_code.sh', 'r') as file:
    script_contents = file.read()

# Use the script_contents in the datacrunch function
datacrunch.startup_scripts.create('cuda_drivers', script_contents)

In [17]:
script_id = datacrunch.startup_scripts.get()[0].id

In [18]:
# Create a new instance
instance = datacrunch.instances.create(instance_type=instance_type,
                                      image='ubuntu-22.04-cuda-12.0-docker',
                                      ssh_key_ids=ssh_keys,
                                      hostname='quint',
                                      description='Transcription and summarization API',
                                      startup_script_id=script_id,
                                      os_volume={
                                        "name": "OS volume",
                                        "size": 100
                                        }
                                      )

In [19]:
import time
while datacrunch.instances.get()[0].status=='provisioning':
    time.sleep(5)
print('Instance is running, you can now connect to it.')

Instance is running, you can now connect to it.


In [21]:
host = datacrunch.instances.get()[0].ip
print(f'ssh root@{host} -i ~/.ssh/{key_filename}')
# Connect to server via ssh and perform command "sudo reboot" to restart server for enabling Nvidia Drivers

ssh root@65.109.75.44 -i ~/.ssh/poloniki_ya_ru


# Possible error WARNING: REMOTE HOST IDENTIFICATION HAS CHANGED!

sed -i '' '47d' /Users/poloniki/.ssh/known_hosts

In [25]:
print('cd quint')

cd quint


In [35]:
!scp -r -i ~/.ssh/$key_filename ../* root@{host}:~/

Dockerfile                                    100%  511     7.7KB/s   00:00    
Dockerfile.jax                                100%  711    10.5KB/s   00:00    
LICENSE                                       100% 1074    11.4KB/s   00:00    
Makefile                                      100%  353     5.2KB/s   00:00    
README.md                                     100% 1127    16.7KB/s   00:00    
podcast_transcript.txt                        100%  102KB 378.7KB/s   00:00    
env.sample                                    100%    0     0.0KB/s   00:00    
logo.png                                      100%   28KB 407.5KB/s   00:00    
app.py                                        100% 7561   108.3KB/s   00:00    
Deploy_gpu_instance.ipynb                     100%   12KB 169.6KB/s   00:00    
Jax_Chunking_IN_DEV.ipynb                     100%   47KB 325.3KB/s   00:00    
Chunking text into paragraphs.ipynb           100%  749KB   2.6MB/s   00:00    
transcriber.py                          

In [139]:
print('docker build -t jax --file Dockerfile.jax .')

docker build -t jax --file Dockerfile.jax .


In [143]:
print('docker run --gpus all -p 80:80 --shm-size=1g --rm jax')

docker run --gpus all -p 80:80 jax
